In [1]:
from pathlib import Path
import regex

# make dataset

In [66]:
search_files = list(Path('./test/searcher_gpt-4/').glob('*.output'))
predictor_files = list(Path('./test/predictor_gpt-4/').glob('*.output'))

In [67]:
with open('test/searcher2.txt') as f:
    search_question = [line.strip() for line in f]

with open('test/predictor2.txt') as f:
    predict_question = [line.strip() for line in f]

In [68]:
def parsing(logfile):
    logfile = regex.sub(r"\[.+?m", "", logfile)
    if '########## Output ##############################' not in logfile:
        return None
    logfile = logfile.split('> Entering new AgentExecutor chain...\n')[-1]
    

    logfile = logfile.split('########## Output ##############################')[0].strip()

    tab = 0
    new_logfile = ''
    for line in logfile.split("\n"):
        if line.startswith('> Entering'):
            tab += 1
        elif line.startswith('> Finished'):
            tab -= 1
        elif not line.strip():
            continue
        elif not tab:
            new_logfile += line + "\n"

    return new_logfile.strip()


In [69]:
system_prompt = """You act like a material scientist answering a question. Answer the following questions as best you can. You have access to the following tools:

search_csv: A tools that extract accurate properties from a look-up table in the database. To use search_csv, the material must be refcode.The input must be a detailed full sentence to answer the question.
predictor: Useful tool to predict material properties using machine learning. More imprecise than the search_csv tool, but can be used universally. You must use predictor before using google_search or wikipedia. The input must be a detailed full sentence to answer the question.
generator: The tool to use when you need to create materials with specific properties. `generator` must only be used when the original question wants to generate. input must be provided in the form of a full sentence. 
visualizer: A tools to visualize the structure. The input must be a detailed full sentence to answer the question.
Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.
ase_repl: A tools for analyzing materials using the ASE library.It allows you to perform tasks using ASE, such as atom coordinates and cell informationThe input must be a detailed full sentence to answer the question.
unit_converter: A tool for converting unit. Ask a question about unit conversion and it will tell you how to convert and what information you need. 
Calculator: Useful for when you need to answer questions about math.
google_search: A wrapper around Google Search. Useful for when you need to answer questions about current events. Input should be a search query.
Google Search Results JSON: A wrapper around Google Search. Useful for when you need to answer questions about current events. Input should be a search query. Output is a JSON array of the query results
Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

You must not use tool `generator` when `Question` does not required to generate. 

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [search_csv, predictor, generator, visualizer, Python_REPL, ase_repl, unit_converter, Calculator, google_search, Google Search Results JSON, Wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""

In [70]:
def make_jsonl(prompt1, prompt2):
    system = {
        "role": "system",
        "content": system_prompt,
    }

    user = {
        "role": "user",
        "content": prompt1.strip()
    }

    assistant = {
        "role": "assistant",
        "content": prompt2.strip()
    }
    tmp = {}
    tmp['messages'] = [system, user, assistant]
    return tmp

In [71]:
total = []

In [75]:
#file_list = predictor_files
#question_list = predict_question
file_list = search_files
question_list = search_question

len = 0

for file in file_list:
    number = int(file.stem.split("_")[0])
    with file.open() as f:
        logfile = f.read()

    new_log = parsing(logfile)
    question = question_list[number]

    #print (question, number, new_log)
    if not new_log:
        continue
    if number in range(11):  # for test set
        print (number)
        continue

    new_log = f'Question: {question}\nThought: ' + new_log

    split = regex.split(r"(Observation:\s?|Thought:\s?)", new_log)

    for i, s in enumerate(split):
        if s.startswith('Thought:'):
            prompt1 = "".join(split[:i+1])
            prompt2 = "".join(split[i+1:i+3])

            total.append(make_jsonl(prompt1, prompt2))

    len += 1

print ('len: ', len)


8
10
5
7
6
1
9
2
3
0
4
len:  83


# Table search tool

In [4]:
search_files[0]

PosixPath('test/searcher_gpt-4/59_.output')

In [21]:
def parsing_table(logfile):

    logfile = regex.sub(r"\[.+?m", "", logfile)
    if '########## Output ##############################' not in logfile:
        return None
    logfile = logfile.split('> Entering new AgentExecutor chain...\n')[-1]
    
    logfile = logfile.split('########## Output ##############################')[0].strip()

    tab = 0
    new_logfile = ''
    for line in logfile.split("\n"):
        if line.startswith('> Entering'):
            tab += 1
        elif line.startswith('> Finished'):
            tab -= 1
        elif not line.strip():
            continue
        elif tab == 1:
            new_logfile += line.replace("[Table Searcher] ", "") + "\n"
        elif tab == 0 and line.startswith('Observation'):
            new_logfile += line.replace('Observation:', 'Final Answer:')

    return new_logfile.strip()

    return logfile

In [33]:
import pandas as pd 

df = pd.read_excel('./database/tables/coremof.xlsx')

In [37]:
system_prompt = """You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should make a valid python command as input. You must use print the output using the `print` function at the end.
You should use the `to_markdown` function when you print a pandas object.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Input: the valid python code only using the Pandas library
Observation: the result of python code
... (this Thought/Input/Observation can repeat N times)
Final Thought: you should think about how to answer the question based on your observation
Final Answer: the final answer to the original input question. If you can't answer the question, say `nothing`

The index of the dataframe must be be one of ['name', 'Largest cavity diameter (Å)', 'Pore limiting diameter (Å)', 'Largest free pore diameter (Å)', 'Density (g/cm^3)', 'Accessible Surface Area (m^2/g)', 'Nonaccessible surface area (m^2/g)', 'Accessible volume volume fraction', 'void fraction', 'Accessible pore volume (cm^3/g)', 'Accessible volume (cm^3/g)', 'Nonaccessible Volume (cm^3/g)', 'Metal type', 'Has open metal site', 'Type of open metal']. If it's not in the index you want, skip straight to Final Thought.

Begin!"""

In [64]:
total_table = []

In [65]:
file_list = search_files
question_list = search_question

len = 0

for file in file_list[:]:
    number = int(file.stem.split("_")[0])
    with file.open() as f:
        logfile = f.read()

    new_log = parsing_table(logfile)
    question = question_list[number]

    #print (question, number, new_log)
    if not new_log:
        continue

    if number in range(11):  # for test set
        print (number)
        continue

    new_log = f'Question: {question}\n' + new_log

    split = regex.split(r"(Observation:\s?|(?:Final )?Thought:\s?)", new_log)

    for i, s in enumerate(split):
        if s.startswith('Thought:') or s.startswith('Final Thought:'):
            prompt1 = "".join(split[:i+1])
            prompt2 = "".join(split[i+1:i+3])

            total_table.append(make_jsonl(prompt1, prompt2))

    len += 1

print ('Len: ', len)

8
10
5
7
6
1
9
2
3
0
4
Len:  83


In [54]:
for i in total_table:

    print ('(!)',i['messages'][1]['content'])
    print ('(!)', i['messages'][2]['content'])
    print ("\n\n")

(!) Question: Can cg901114e_si_002 have an open metal site?
Thought:
(!) To find the information about a specific material, we can use the loc function in pandas. This function allows us to access a group of rows and columns by label(s) or a boolean array.
Input: print(df.loc[df['name'] == 'cg901114e_si_002'].to_markdown())
Observation:



(!) Question: Can cg901114e_si_002 have an open metal site?
Thought: To find the information about a specific material, we can use the loc function in pandas. This function allows us to access a group of rows and columns by label(s) or a boolean array.
Input: print(df.loc[df['name'] == 'cg901114e_si_002'].to_markdown())
Observation: 
|      | name             |   Largest cavity diameter (Å) |   Pore limiting diameter (Å) |   Largest free pore diameter (Å) |   Density (cm^3/g) |   Accessible Surface Area (m^2/cm^3) |   Accessible Surface Area (m^2/g) |   Nonaccessible Surface Area (m^2/cm^3) |   Nonaccessible surface area (m^2/g) |   Accessible volume

# write jsonl

In [61]:
import json

In [123]:
for item in total:
    with open('../experimental/ft_agent/ft_agent.jsonl', 'a') as f:
        json.dump(item, f)
        f.write('\n')

In [143]:
print ('total dataset: ', len(total))

total dataset:  432


In [62]:
for item in total_table:
    with open('../experimental/ft_agent/ft_table.jsonl', 'a') as f:
        json.dump(item, f)
        f.write('\n')

In [63]:
print ('total dataset: ', len(total_table))

total dataset:  182
